# "COVID-19 en Chile: Estadísticas demográficas"
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por sexo y rango de edad.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [2]:
#hide
import numpy as np
import pandas as pd
import altair as alt

# Evolución de casos confirmados por sexo

In [3]:
#hide
cases_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' \
    + 'master/output/producto16/CasosGeneroEtario.csv'
)

In [4]:
#hide
today = cases_raw.iloc[:,-1].name
update_date = pd.to_datetime(today)
today

'2020-06-23'

In [5]:
#hide
cases_sex = cases_raw.groupby('Sexo').sum()
cases_sex = cases_sex.reset_index().melt('Sexo', var_name="Fecha (año-mes-día)", value_name="Casos confirmados")

In [6]:
#hide
cases_sex = cases_sex.set_index("Fecha (año-mes-día)")
cases_sex["Porcentaje"] = cases_sex["Casos confirmados"]/cases_raw.sum().drop(["Grupo de edad", "Sexo"])
cases_sex = cases_sex.reset_index()

In [7]:
#hide_input
input_dropdown = alt.binding_select(options=cases_sex['Sexo'].unique())
selection1 = alt.selection_single(fields=['Sexo'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Sexo'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Sexo:N', scale=alt.Scale(scheme='tableau10'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(cases_sex).mark_bar().encode(
    x=alt.X("Fecha (año-mes-día)", axis=alt.Axis(title="")),
    y=alt.Y("Casos confirmados"),
    tooltip = ["Fecha (año-mes-día)", "Casos confirmados", "Porcentaje"],
    color=color
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(cases_sex).mark_point().encode(
    y=alt.Y('Sexo:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

bars2 = alt.Chart(cases_sex).mark_bar().encode(
    x=alt.X("Fecha (año-mes-día)", axis=alt.Axis(title="Fecha (año-mes-día)", ticks=False, labels=False)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="Normalizados", format='%'), stack="normalize"),
    color=color,
    tooltip = ["Fecha (año-mes-día)", "Casos confirmados", "Porcentaje"],
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

alt.vconcat(
    bars1.properties(
        title = 'COVID-19 en Chile: Evolución de casos confirmados por sexo',
        width=600) | legend,
    bars2.properties(
        height = 80,
        width=600)
).configure_view(
    stroke=None
).configure_concat(
    spacing=1
)

alt.VConcatChart(...)

In [8]:
#hide
censo_raw = pd.read_csv(
    'https://raw.githubusercontent.com/alonsosilvaallende/COVID-19/master/data/Censo_2017_Chile.csv',
    thousands='.'
)

In [9]:
#hide
censo_raw["Hombre"].sum(), censo_raw["Mujer"].sum(), censo_raw["Hombre"].sum()/(censo_raw["Hombre"].sum()+censo_raw["Mujer"].sum())

(8601989, 8972014, 0.48947237575867036)

In [10]:
#hide
d = {'M': censo_raw["Hombre"].sum(), 'F': censo_raw["Mujer"].sum()}
d

{'M': 8601989, 'F': 8972014}

In [11]:
#hide
censo_sex = pd.DataFrame(d, index=['Sex'])

# Casos confirmados por grupo de edad

In [12]:
#hide
cases_raw = cases_raw.groupby('Grupo de edad').sum()

In [13]:
#hide
cases = pd.DataFrame()
cases['Grupo de edad'] = ['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']

In [14]:
#hide
cases["Casos confirmados"] = [
    cases_raw.iloc[:8,-1].sum(),
    cases_raw.iloc[8:10,-1].sum(),
    cases_raw.iloc[10:12,-1].sum(),
    cases_raw.iloc[12:14,-1].sum(),
    cases_raw.iloc[14:16,-1].sum(),
    cases_raw.iloc[16:,-1].sum()
]

In [15]:
#hide
total_cases = cases["Casos confirmados"].sum()
total_cases

255361

In [16]:
#hide
cases["Porcentaje"] = np.round(100*cases["Casos confirmados"]/total_cases, decimals=1)

In [17]:
#hide
cases["Texto"] = \
[f"{cases['Casos confirmados'].iloc[i]} ({cases['Porcentaje'].iloc[i]}%)" for i, val in cases.iterrows()]

In [18]:
#hide_input
bars = alt.Chart(cases.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Casos confirmados'),
    y = alt.Y('Grupo de edad', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80']),
    tooltip = ['Grupo de edad', 'Casos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Casos confirmados por grupo de edad al {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Fallecidos confirmados por grupo de edad

In [19]:
#hide
deaths_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' + \
    'master/output/producto10/FallecidosEtario.csv'
)

In [20]:
#hide
grupo_de_edad = ['<=39', '40-49', '50-59', '60-69', '70-79', '80-89', '>=90']

In [21]:
#hide
deaths = deaths_raw[['Grupo de edad', today]].copy()

In [22]:
#hide
total_deaths = deaths[today].sum()
total_deaths

4505

In [23]:
#hide
deaths = deaths.rename(columns={today: "Fallecidos confirmados"})

In [24]:
#hide
deaths['Porcentaje'] = np.round(100*deaths["Fallecidos confirmados"]/total_deaths, decimals=1)

In [25]:
#hide
deaths['Texto'] = \
[f"{deaths['Fallecidos confirmados'].iloc[i]} ({deaths['Porcentaje'].iloc[i]}%)" for i, val in deaths.iterrows()]

In [26]:
#hide_input
bars = alt.Chart(deaths.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Fallecidos confirmados'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Fallecidos confirmados', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Fallecidos confirmados por grupo de edad al {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Letalidad por grupo de edad

In [27]:
#hide
data = cases.drop(columns=["Porcentaje", "Texto"])

In [28]:
#hide
a = list(deaths["Fallecidos confirmados"][:5])
a

[98, 150, 434, 942, 1344]

In [29]:
#hide
a.append(deaths["Fallecidos confirmados"][5:].sum())
a

[98, 150, 434, 942, 1344, 1537]

In [30]:
#hide
data["Fallecidos confirmados"] = a

In [31]:
#hide
data["Letalidad"] = 100*data["Fallecidos confirmados"]/data["Casos confirmados"]

In [32]:
#hide
data['Texto'] = \
[f"{data['Letalidad'].iloc[i]:.2f}" for i, val in data.iterrows()]

In [33]:
#hide_input
bars = alt.Chart(data.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Letalidad'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Casos confirmados', 'Fallecidos confirmados', 'Letalidad'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Covid-19 en Chile: Letalidad por grupo de edad al {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

In [34]:
#hide
100*deaths["Fallecidos confirmados"].sum()/cases["Casos confirmados"].sum()

1.7641691566057465

# Población por grupo de edad

In [35]:
#hide
censo = pd.DataFrame()
censo['Grupo de edad'] = censo_raw['Grupo de edad']

In [36]:
#hide
censo['Poblacion'] = censo_raw.drop(columns='Grupo de edad').sum(axis=1)

In [37]:
#hide
new_censo = pd.DataFrame()
new_censo['Grupo de edad'] = grupo_de_edad

In [38]:
#hide
new_censo["Poblacion"] = \
[censo.iloc[:8]["Poblacion"].sum(), \
censo.iloc[8:10]["Poblacion"].sum(), \
censo.iloc[10:12]["Poblacion"].sum(), \
censo.iloc[12:14]["Poblacion"].sum(), \
censo.iloc[14:16]["Poblacion"].sum(), \
censo.iloc[16:18]["Poblacion"].sum(), \
censo.iloc[18:]["Poblacion"].sum()]

In [39]:
#hide
total_poblacion = new_censo["Poblacion"].sum()
total_poblacion

17574003

In [40]:
#hide
new_censo["Porcentaje"] = np.round(100*new_censo["Poblacion"]/total_poblacion, decimals=1)

In [41]:
#hide
new_censo['Texto'] = \
[f"{format(new_censo['Poblacion'].iloc[i],',').replace(',', '.')} ({new_censo['Porcentaje'].iloc[i]}%)" for i, val in new_censo.iterrows()]

In [42]:
#hide_input
bars = alt.Chart(new_censo.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Poblacion', axis=alt.Axis(title="Población")),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    tooltip = ['Grupo de edad', 'Poblacion', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None),
    order = alt.Order('index:O')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Población Censo 2017",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Pacientes UCI por rango de edad

In [43]:
#hide
pacientes_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/'
    + 'master/output/producto9/HospitalizadosUCIEtario.csv'
)

In [44]:
#hide
pacientes = pacientes_raw[['Grupo de edad', today]]

In [45]:
#hide
total_pacientes = pacientes[today].sum()
total_pacientes

2009

In [46]:
#hide
pacientes = pacientes.rename(columns={today: "Pacientes UCI"})

In [47]:
#hide
pacientes['Porcentaje'] = np.round(100*pacientes["Pacientes UCI"]/total_pacientes, decimals=1)

In [48]:
#hide
pacientes['Texto'] = \
[f"{pacientes['Pacientes UCI'].iloc[i]} ({pacientes['Porcentaje'].iloc[i]}%)" for i, val in pacientes.iterrows()]

In [49]:
#hide_input
bars = alt.Chart(pacientes.reset_index()).mark_bar(opacity=0.8, size=30).encode(
    x = alt.X('Pacientes UCI'),
    y = alt.Y('Grupo de edad', sort=grupo_de_edad),
    order = alt.Order('index:O'),
    tooltip = ['Grupo de edad', 'Pacientes UCI', 'Porcentaje'],
    color = alt.Color('Grupo de edad', scale=alt.Scale(scheme='category10'), legend=None)
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Pacientes UCI presentes el {pd.to_datetime(today).strftime('%d/%m/%Y')}",
    width = 600,
    height = alt.Step(40)
) + text

alt.LayerChart(...)

# Evolución de casos confirmados por rango de edad

In [50]:
#hide
cases_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/' \
    + 'master/output/producto16/CasosGeneroEtario.csv'
)

In [51]:
#hide
cases = pd.DataFrame()
cases['<=39'] = cases_raw.groupby('Grupo de edad').sum().iloc[:8].sum()
cases['40-49'] = cases_raw.groupby('Grupo de edad').sum().iloc[8:10].sum()
cases['50-59'] = cases_raw.groupby('Grupo de edad').sum().iloc[10:12].sum()
cases['60-69'] = cases_raw.groupby('Grupo de edad').sum().iloc[12:14].sum()
cases['70-79'] = cases_raw.groupby('Grupo de edad').sum().iloc[14:16].sum()
cases['>=80'] = cases_raw.groupby('Grupo de edad').sum().iloc[16:].sum()

In [52]:
#hide
cases = cases.T.reset_index()
cases = cases.rename(columns={'index': 'Grupo de edad'})

In [53]:
#hide
cases = cases.reset_index().melt(['index', 'Grupo de edad'], var_name="fecha (año-mes-día)", value_name="casos confirmados")

In [54]:
#hide_input
input_dropdown = alt.binding_select(options=data['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(cases).mark_bar().encode(
    x = 'fecha (año-mes-día):N',
    y = 'casos confirmados:Q',
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'casos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(cases).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de casos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

In [55]:
#hide_input
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(cases).mark_area().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('casos confirmados:Q', stack="normalize"),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'casos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection2
)

legend = alt.Chart(cases).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de casos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

# Evolución de fallecidos confirmados por rango de edad

In [56]:
#hide
data = deaths_raw.reset_index().melt(['index', 'Grupo de edad'])

In [57]:
#hide
data = data.rename(columns={'variable': "fecha (año-mes-día)", 'value': "fallecidos confirmados"})

In [58]:
#hide_input
input_dropdown = alt.binding_select(options=data['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_bar().encode(
    x = 'fecha (año-mes-día):N',
    y = 'fallecidos confirmados:Q',
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'fallecidos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de fallecidos confirmados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

In [59]:
#hide_input
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_area().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('fallecidos confirmados:Q', stack="normalize", sort=grupo_de_edad),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'fallecidos confirmados'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'index',
    sort='descending'
    )
).add_selection(
    selection2
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('Grupo de edad',  axis=alt.Axis(orient='right'), sort=grupo_de_edad),
    color=color
).add_selection(
    selection2
)

bars.properties(
    title = 'COVID-19 en Chile: Evolución de fallecidos confirmados normalizados por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

# Evolución de la letalidad por rango de edad

In [60]:
#hide
deaths = deaths_raw.iloc[:5]
deaths = deaths.append(deaths_raw.iloc[5:].sum(), ignore_index=True)
deaths.loc[5, "Grupo de edad"] = ">=80"

In [61]:
#hide
cases = pd.DataFrame()
cases['<=39'] = cases_raw.groupby('Grupo de edad').sum().iloc[:8].sum()
cases['40-49'] = cases_raw.groupby('Grupo de edad').sum().iloc[8:10].sum()
cases['50-59'] = cases_raw.groupby('Grupo de edad').sum().iloc[10:12].sum()
cases['60-69'] = cases_raw.groupby('Grupo de edad').sum().iloc[12:14].sum()
cases['70-79'] = cases_raw.groupby('Grupo de edad').sum().iloc[14:16].sum()
cases['>=80'] = cases_raw.groupby('Grupo de edad').sum().iloc[16:].sum()

In [62]:
#hide
cases = cases.T.reset_index()
cases = cases.rename(columns={'index': 'Grupo de edad'})

In [63]:
#hide
deaths = deaths.set_index("Grupo de edad").T
cases = cases.set_index("Grupo de edad").T
letalidad = deaths.divide(cases['2020-04-09':])

In [64]:
#hide
cases.tail(3)

Grupo de edad,<=39,40-49,50-59,60-69,70-79,>=80
2020-06-15,92142,31008,28049,15780,7737,4714
2020-06-19,119298,39668,36128,20250,9905,6103
2020-06-23,131528,43710,39940,22457,10922,6804


In [65]:
#hide
deaths.tail(3)

Grupo de edad,<=39,40-49,50-59,60-69,70-79,>=80
2020-06-24,105,157,458,987,1414,1610
2020-06-25,107,160,473,1035,1460,1668
2020-06-26,109,162,499,1071,1511,1716


In [66]:
#hide
letalidad = letalidad.reset_index().melt("index")
letalidad = letalidad.rename(columns={"index": "fecha (año-mes-día)", "value": "letalidad"})
letalidad = letalidad.dropna()

In [67]:
#hide
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

input_dropdown = alt.binding_select(options=letalidad['Grupo de edad'].unique())
selection1 = alt.selection_single(fields=['Grupo de edad'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Grupo de edad'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None),
                    alt.value('lightgray'))


base = alt.Chart(letalidad).mark_line().encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('letalidad:Q', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80'], axis=alt.Axis(format='%')),
    color = color,
    tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'letalidad'],
).add_selection(
    label, selection1, selection2
).transform_filter(
    selection1
)

chart = alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('fecha (año-mes-día):N', axis=alt.Axis(title='fecha (año-mes-día):N'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),
    
    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    data=letalidad
)

legend = alt.Chart(letalidad).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(
    title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
    width = 600
) | legend

alt.HConcatChart(...)

In [68]:
#hide_input
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

base = alt.Chart(letalidad).mark_line(point=True).encode(
    x = 'fecha (año-mes-día):N',
    y = alt.Y('letalidad:Q', sort=['<=39', '40-49', '50-59', '60-69', '70-79', '>=80'], axis=alt.Axis(format='%')),
    color = alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None),
    #tooltip = ['fecha (año-mes-día)', 'Grupo de edad', 'letalidad'],
)

legend = alt.Chart(letalidad).mark_point().encode(
    y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
    color=alt.Color('Grupo de edad:N', scale=alt.Scale(scheme='tableau10'), legend=None)
)

alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x = alt.X('fecha (año-mes-día):N', axis=alt.Axis(title='fecha (año-mes-día)'), sort=None)
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),
    
    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='letalidad:Q'
    ).transform_filter(label),
    
    data=letalidad
).properties(
    title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
    width = 600
) | legend

# legend = alt.Chart(letalidad).mark_point().encode(
#     y=alt.Y('Grupo de edad', axis=alt.Axis(orient='right'), sort=['>=80','70-79','60-69','50-59','40-49','<=39']),
#     color=color
# )

# chart.properties(
#     title = 'COVID-19 en Chile: Evolución de la letalidad por rango de edad',
#     width = 600
# ) | legend

alt.HConcatChart(...)